# 🔍 Edge AI Prototype: Recyclable Item Classification with TensorFlow Lite

## 🎯 Objective
- Train a lightweight CNN model to classify recyclable items.
- Convert to TensorFlow Lite for edge deployment.
- Test on sample images and report performance.
- Highlight Edge AI advantages in real-time applications.

---


In [4]:
# 📦 Install necessary libraries
!pip install tensorflow tensorflow-datasets matplotlib
!pip install tensorflow

import tensorflow as tf
import numpy as np
import matplotlib.pyplot as plt
import os
from tensorflow.keras import layers, models
from tensorflow.keras.preprocessing.image import ImageDataGenerator


ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow
ERROR: Could not find a version that satisfies the requirement tensorflow (from versions: none)
ERROR: No matching distribution found for tensorflow


ModuleNotFoundError: No module named 'tensorflow'

In [ ]:
# 📁 Load dataset (e.g., TrashNet or custom)
# Here we simulate by using the tf_flowers dataset for demonstration
import tensorflow_datasets as tfds

(dataset_train, dataset_val), dataset_info = tfds.load(
    'tf_flowers',
    split=['train[:80%]', 'train[80%:]'],
    with_info=True,
    as_supervised=True
)

IMG_SIZE = (64, 64)
BATCH_SIZE = 32

def format_img(image, label):
    image = tf.image.resize(image, IMG_SIZE) / 255.0
    return image, label

train = dataset_train.map(format_img).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)
val = dataset_val.map(format_img).batch(BATCH_SIZE).prefetch(tf.data.AUTOTUNE)


In [ ]:
# 🧠 Define a compact CNN model
model = models.Sequential([
    layers.Input(shape=(64, 64, 3)),
    layers.Conv2D(16, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Conv2D(32, 3, activation='relu'),
    layers.MaxPooling2D(),
    layers.Flatten(),
    layers.Dense(64, activation='relu'),
    layers.Dense(dataset_info.features['label'].num_classes, activation='softmax')
])

model.compile(optimizer='adam',
              loss='sparse_categorical_crossentropy',
              metrics=['accuracy'])

model.summary()


In [ ]:
# 🚀 Train the model
history = model.fit(train, validation_data=val, epochs=5)


In [ ]:
# 💾 Save the model
model.save("recycle_model")


In [ ]:
# 🔄 Convert to TensorFlow Lite
converter = tf.lite.TFLiteConverter.from_saved_model("recycle_model")
tflite_model = converter.convert()

with open("recycle_model.tflite", "wb") as f:
    f.write(tflite_model)


In [ ]:
# 🔍 Load and run inference with the TFLite model
interpreter = tf.lite.Interpreter(model_path="recycle_model.tflite")
interpreter.allocate_tensors()

input_details = interpreter.get_input_details()
output_details = interpreter.get_output_details()

# Prepare a test image
for image, label in val.take(1):
    img = image[0:1].numpy().astype(np.float32)

interpreter.set_tensor(input_details[0]['index'], img)
interpreter.invoke()
output = interpreter.get_tensor(output_details[0]['index'])

print("Predicted label:", np.argmax(output))


## 📊 Accuracy & Deployment Summary

| Metric              | Value (Example) |
|---------------------|-----------------|
| Validation Accuracy | ~91%            |
| Model Size (TFLite) | ~200 KB         |
| Inference Time      | ~30ms/image     |

---

## ⚡ Benefits of Edge AI in Real-Time Systems

- **Low Latency**: Local inference = faster decisions.
- **Bandwidth Efficient**: No need for constant cloud access.
- **Privacy Preserving**: Sensitive image data stays local.
- **Resilient**: Works offline—ideal for remote deployments.
